# Heart Failure Prediction

This notebook showcases heart failure prediction on a [popular Kaggle dataset](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction) using CyclOps. The task is formulated as a binary classification task, where we predict the probability that a patient will have heart failure.

## Import Libraries

In [ ]:
import copy
import inspect
import os
import shutil
from datetime import date
from pathlib import Path

import numpy as np
import plotly.express as px
from datasets import Dataset
from datasets.features import ClassLabel
from dateutil.relativedelta import relativedelta
from kaggle.api.kaggle_api_extended import KaggleApi
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from cyclops.data.slicer import SliceSpec
from cyclops.evaluate.fairness import FairnessConfig  # noqa: E402
from cyclops.evaluate.metrics import MetricCollection, create_metric
from cyclops.models.catalog import create_model
from cyclops.process.feature.feature import TabularFeatures
from cyclops.report import ModelCardReport
from cyclops.report.plot.classification import ClassificationPlotter
from cyclops.report.utils import flatten_results_dict, get_metrics_trends
from cyclops.tasks.mortality_prediction import MortalityPredictionTask
from cyclops.utils.file import join, load_dataframe

CyclOps offers a package for documentation of the model through a model card. The `ModelCardReport` class is used to populate and generate the model card as an HTML file. The model card has the following sections:
- Model Details: This section contains descriptive metadata about the model such as the owners, version, license, etc.
- Model Parameters: This section contains the technical details of the model such as the model architecture, training parameters, etc.
- Considerations: This section contains descriptions of the considerations involved in developing and using the model such as the intended use, limitations, etc.
- Quantitative Analysis: This section contains the performance metrics of the model for different sets of the data and subpopulations.
- Explainaibility Analysis: This section contains the explainability metrics of the model.
- Fairness Analysis: This section contains the fairness metrics of the model.

We will use this to document the model development process as we go along and generate the model card at the end.

`The model card tool is a work in progress and is subject to change.`

In [ ]:
report = ModelCardReport()

## Constants

In [ ]:
DATA_DIR = "./data"
RANDOM_SEED = 85
NAN_THRESHOLD = 0.75
TRAIN_SIZE = 0.8

## Data Loading

Before starting, make sure to install the Kaggle API by running `pip install kaggle`. To use the Kaggle API, you need to sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (`https://www.kaggle.com/<username>/account`) and select 'Create API Token'. This will trigger the download of kaggle.json, a file containing your API credentials. Place this file in the location `~/.kaggle/kaggle.json` on your machine.

In [ ]:
api = KaggleApi()
api.authenticate()
api.dataset_download_files(
    "fedesoriano/heart-failure-prediction", path=DATA_DIR, unzip=True
)

In [ ]:
df = load_dataframe(join(DATA_DIR, "heart.csv"), file_format="csv")
df = df.reset_index()
df["ID"] = df.index
df

### Sex values

In [ ]:
fig = px.pie(df, names="Sex")

fig.update_layout(
    title="Sex Distribution",
)

fig.show()

**Add the figure to the report**

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Sex Distribution",
    section_name="datasets",
)

###  Age distribution

In [ ]:
fig = px.histogram(df, x="Age")
fig.update_layout(
    title="Age Distribution",
    xaxis_title="Age",
    yaxis_title="Count",
    bargap=0.2,
)

fig.show()

**Add the figure to the report**

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Age Distribution",
    section_name="datasets",
)

### Outcome distribution

In [ ]:
df["outcome"] = df["HeartDisease"].astype("int")
df = df.drop(columns=["HeartDisease"])

In [ ]:
fig = px.pie(df, names="outcome")
fig.update_traces(textinfo="percent+label")
fig.update_layout(title_text="Outcome Distribution")
fig.update_traces(
    hovertemplate="Outcome: %{label}<br>Count: \
    %{value}<br>Percent: %{percent}"
)
fig.show()

**Add the figure to the report**

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Outcome Distribution",
    section_name="datasets",
)

In [ ]:
class_counts = df["outcome"].value_counts()
class_ratio = class_counts[0] / class_counts[1]
class_ratio

In [ ]:
df.columns

From all the features in the dataset, we select 20 of them which was reported by [Li et al.](https://pubmed.ncbi.nlm.nih.gov/34301649/)  to be the most important features in this classification task. 

In [ ]:
features_list = [
    "Age",
    "Sex",
    "ChestPainType",
    "RestingBP",
    "Cholesterol",
    "FastingBS",
    "RestingECG",
    "MaxHR",
    "ExerciseAngina",
    "Oldpeak",
    "ST_Slope",
]

features_list = sorted(features_list)

### Identifying feature types

Cyclops `TabularFeatures` class helps to identify feature types, an essential step before preprocessing the data. Understanding feature types (numerical/categorical/binary) allows us to apply appropriate preprocessing steps for each type.

In [ ]:
tab_features = TabularFeatures(
    data=df.reset_index(),
    features=features_list,
    by="ID",
    targets="outcome",
)
tab_features.types

### Creating data preprocessors

We create a data preprocessor using sklearn's ColumnTransformer. This helps in applying different preprocessing steps to different columns in the dataframe. For instance, binary features might be processed differently from numeric features.

In [ ]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())]
)

binary_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent"))]
)

In [ ]:
numeric_features = sorted((tab_features.features_by_type("numeric")))
numeric_indices = [
    df[features_list].columns.get_loc(column) for column in numeric_features
]
numeric_features

In [ ]:
binary_features = sorted(tab_features.features_by_type("binary"))
binary_features.remove("outcome")
ordinal_features = sorted(tab_features.features_by_type("ordinal"))
binary_indices = [
    df[features_list].columns.get_loc(column) for column in binary_features
]
ordinal_indices = [
    df[features_list].columns.get_loc(column) for column in ordinal_features
]
binary_features, ordinal_features

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_indices),
        (
            "onehot",
            OneHotEncoder(handle_unknown="ignore"),
            binary_indices + ordinal_indices,
        ),
    ],
    remainder="passthrough",
)

Let's document the dataset in the model card. This can be done using the `log_dataset` method, which takes the following arguments:
- description: A description of the dataset.
- citation: The citation for the dataset.
- link: A link to a resource for the dataset.
- license_id: The SPDX license identifier for the dataset.
- version: The version of the dataset.
- features: A list of features in the dataset.
- split: The split of the dataset (train, test, validation, etc.).
- sensitive_features: A list of sensitive features used to train/evaluate the model.
- sensitive_feature_justification: A justification for the sensitive features used to train/evaluate the model.

In [ ]:
report.log_dataset(
    description="""This dataset was created by combining different datasets
    already available independently but not combined before. In this dataset,
    5 heart datasets are combined over 11 common features. Every dataset used
    can be found under the Index of heart disease datasets from UCI
    Machine Learning Repository on the following link:
    https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/.""",
    citation=inspect.cleandoc(
        """
        @misc{fedesoriano,
          title={Heart Failure Prediction Dataset.},
          author={Fedesoriano, F},
          year={2021},
          publisher={Kaggle}
        }
    """
    ),
    link="""
    https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction
    """,
    license_id="CC0-1.0",
    version="Version 1",
    features=features_list,
    sensitive_features=["Sex", "Age"],
    sensitive_feature_justification="Demographic information like age and gender \
        often have a strong correlation with health outcomes. For example, older \
        patients are more likely to have a higher risk of heart disease.",
)

## Creating Hugging Face Dataset

We convert our processed Pandas dataframe into a Hugging Face dataset, a powerful and easy-to-use data format which is also compatible with CyclOps models and evaluator modules. The dataset is then split to train and test sets.

In [ ]:
dataset = Dataset.from_pandas(df)
dataset.cleanup_cache_files()
dataset

In [ ]:
dataset = dataset.cast_column("outcome", ClassLabel(num_classes=2))
dataset = dataset.train_test_split(
    train_size=TRAIN_SIZE, stratify_by_column="outcome", seed=RANDOM_SEED
)

## Model Creation

CyclOps model registry allows for straightforward creation and selection of models. This registry maintains a list of pre-configured models, which can be instantiated with a single line of code. Here we use a SGD classifier to fit a logisitic regression model. The model configurations can be passed to `create_model` based on the sklearn parameters for SGDClassifer.

In [ ]:
model_name = "sgd_classifier"
model = create_model(model_name, random_state=123, verbose=0, class_weight="balanced")

## Task Creation

We use Cyclops tasks to define our model's task (in this case, MortalityPrediction), train the model, make predictions, and evaluate performance. Cyclops task classes encapsulate the entire ML pipeline into a single, cohesive structure, making the process smooth and easy to manage.

In [ ]:
mortality_task = MortalityPredictionTask(
    {model_name: model}, task_features=features_list, task_target="outcome"
)

In [ ]:
mortality_task.list_models()

## Training

If `best_model_params` is passed to the `train` method, the best model will be selected after the hyperparameter search. The parameters in `best_model_params` indicate the values to create the parameters grid.

Note that the data preprocessor needs to be passed to the tasks methods if the Hugging Face dataset is not already preprocessed. 

In [ ]:
best_model_params = {
    "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
    "learning_rate": ["constant", "optimal", "invscaling", "adaptive"],
    "eta0": [0.1, 0.01, 0.001, 0.0001],
    "metric": "roc_auc",
    "method": "grid",
}

mortality_task.train(
    dataset["train"],
    model_name=model_name,
    transforms=preprocessor,
    best_model_params=best_model_params,
)

In [ ]:
model_params = mortality_task.list_models_params()[model_name]
model_params

**Log the model parameters to the report.**

We can add model parameters to the model card using the `log_model_parameters` method.

In [ ]:
report.log_model_parameters(params=model_params)

## Prediction

The prediction output can be either the whole Hugging Face dataset with the prediction columns added to it or the single column containing the predicted values.

In [ ]:
y_pred = mortality_task.predict(
    dataset["test"],
    model_name=model_name,
    transforms=preprocessor,
    proba=False,
    only_predictions=True,
)
len(y_pred)

## Evaluation

Evaluation is done using various evaluation metrics that provide different perspectives on the model's predictive abilities i.e. standard performance metrics and fairness metrics.

The standard performance metrics can be created using the `MetricCollection` object.

In [ ]:
metric_names = [
    "accuracy",
    "precision",
    "recall",
    "f1_score",
    "auroc",
    "roc_curve",
    "precision_recall_curve",
]
metrics = [create_metric(metric_name, task="binary") for metric_name in metric_names]
metric_collection = MetricCollection(metrics)

In addition to overall metrics, it might be interesting to see how the model performs on certain subpopulations. We can define these subpopulations using `SliceSpec` objects. 

In [ ]:
spec_list = [
    {
        "Age": {
            "min_value": 30,
            "max_value": 50,
            "min_inclusive": True,
            "max_inclusive": False,
        }
    },
    {
        "Age": {
            "min_value": 50,
            "max_value": 70,
            "min_inclusive": True,
            "max_inclusive": False,
        }
    },
]
slice_spec = SliceSpec(spec_list)

A `MetricCollection` can also be defined for the fairness metrics.

In [ ]:
specificity = create_metric(
    metric_name="specificity",
    task="binary",
)
sensitivity = create_metric(
    metric_name="sensitivity",
    task="binary",
)

fpr = 1 - specificity
fnr = 1 - sensitivity

ber = (fpr + fnr) / 2

fairness_metric_collection = MetricCollection(
    {
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "BER": ber,
    }
)

The FairnessConfig helps in setting up and evaluating the fairness of the model predictions.

In [ ]:
fairness_config = FairnessConfig(
    metrics=fairness_metric_collection,
    dataset=None,  # dataset is passed from the evaluator
    target_columns=None,  # target columns are passed from the evaluator
    groups=["Age"],
    group_bins={"Age": [40, 50]},
    group_base_values={"Age": 40},
    thresholds=[0.5],
)

The evaluate methods outputs the evaluation results and the Hugging Face dataset with the predictions added to it.

In [ ]:
results, dataset_with_preds = mortality_task.evaluate(
    dataset["test"],
    metric_collection,
    model_names=model_name,
    transforms=preprocessor,
    prediction_column_prefix="preds",
    slice_spec=slice_spec,
    batch_size=32,
    fairness_config=fairness_config,
    override_fairness_metrics=False,
)
dataset_with_preds

**Log the performance metrics to the report.**

We can add a performance metric to the model card using the `log_performance_metric` method, which expects a dictionary where the keys are in the following format: `slice_name/metric_name`. For instance, `overall/accuracy`. 

We first need to process the evaluation results to get the metrics in the right format.

In [ ]:
results_flat = flatten_results_dict(
    results=results,
    remove_metrics=["BinaryROCCurve", "BinaryPrecisionRecallCurve"],
    model_name=model_name,
)
results_flat

In [ ]:
for name, metric in results_flat.items():
    split, name = name.split("/")
    if name == "BinaryAUROC":
        report.log_quantitative_analysis(
            "performance",
            name=name,
            value=metric,
            metric_slice=split,
            pass_fail_thresholds=0.8,
            pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
        )
    elif name == "BinaryAccuracy":
        report.log_quantitative_analysis(
            "performance",
            name=name,
            value=metric,
            metric_slice=split,
            pass_fail_thresholds=0.6,
            pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
        )
    elif name == "BinaryPrecision":
        report.log_quantitative_analysis(
            "performance",
            name=name,
            value=metric,
            metric_slice=split,
            pass_fail_thresholds=0.6,
            pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
        )
    elif name == "BinaryRecall":
        report.log_quantitative_analysis(
            "performance",
            name=name,
            value=metric,
            metric_slice=split,
            pass_fail_thresholds=0.6,
            pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
        )
    elif name == "BinaryF1Score":
        report.log_quantitative_analysis(
            "performance",
            name=name,
            value=metric,
            metric_slice=split,
            pass_fail_thresholds=0.6,
            pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
        )

We can also use the `ClassificationPlotter` to plot the performance metrics and the add the figure to the model card using the `log_plotly_figure` method.

In [ ]:
plotter = ClassificationPlotter(task_type="binary", class_names=["0", "1"])
plotter.set_template("plotly_white")

In [ ]:
# extracting the ROC curves and AUROC results for all the slices
roc_curves = {
    slice_name: slice_results["BinaryROCCurve"]
    for slice_name, slice_results in results[model_name].items()
}
aurocs = {
    slice_name: slice_results["BinaryAUROC"]
    for slice_name, slice_results in results[model_name].items()
}
roc_curves.keys()

In [ ]:
# plotting the ROC curves for all the slices
roc_plot = plotter.roc_curve_comparison(roc_curves, aurocs=aurocs)
report.log_plotly_figure(
    fig=roc_plot,
    caption="ROC Curve for Female Patients",
    section_name="quantitative analysis",
)
roc_plot.show()

In [ ]:
# Extracting the overall classification metric values.
overall_performance = {
    metric_name: metric_value
    for metric_name, metric_value in results[model_name]["overall"].items()
    if metric_name not in ["BinaryROCCurve", "BinaryPrecisionRecallCurve"]
}

In [ ]:
# Plotting the overall classification metric values.
overall_performance_plot = plotter.metrics_value(
    overall_performance, title="Overall Performance"
)
report.log_plotly_figure(
    fig=overall_performance_plot,
    caption="Overall Performance",
    section_name="quantitative analysis",
)
overall_performance_plot.show()

In [ ]:
# Extracting the metric values for all the slices.
slice_metrics = {
    slice_name: {
        metric_name: metric_value
        for metric_name, metric_value in slice_results.items()
        if metric_name not in ["BinaryROCCurve", "BinaryPrecisionRecallCurve"]
    }
    for slice_name, slice_results in results[model_name].items()
}

In [ ]:
# Plotting the metric values for all the slices.
slice_metrics_plot = plotter.metrics_comparison_bar(slice_metrics)
report.log_plotly_figure(
    fig=slice_metrics_plot,
    caption="Slice Metric Comparison",
    section_name="quantitative analysis",
)
slice_metrics_plot.show()

In [ ]:
# Reformating the fairness metrics
fairness_results = copy.deepcopy(results["fairness"])
fairness_metrics = {}
# remove the group size from the fairness results and add it to the slice name
for slice_name, slice_results in fairness_results.items():
    group_size = slice_results.pop("Group Size")
    fairness_metrics[f"{slice_name} (Size={group_size})"] = slice_results

In [ ]:
# Plotting the fairness metrics
fairness_plot = plotter.metrics_comparison_scatter(
    fairness_metrics, title="Fairness Metrics"
)
report.log_plotly_figure(
    fig=fairness_plot,
    caption="Fairness Metrics",
    section_name="fairness analysis",
)
fairness_plot.show()

### Performance over time

We can monitor the change of performance metrics over time by leveraging historical reports. This involves using the `get_metrics_trends` function to gather prior metrics and merge them with recent results. We can specify which metrics and slices we wish to observe. Once the data is collected, we generate the plot using the `metrics_trends` method from the plotter, which can then be integrated into the report.

Please note, for the purpose of this tutorial, we will create three dummy reports to demonstrate the process of plotting these metric trends.

In [ ]:
# Generating dummy reports
dummy_report_num = 3
dummy_report_dir = join(os.getcwd(), "dummy_reports")

for i in range(dummy_report_num):
    # Create a dummy model card report
    dummy_report = ModelCardReport(output_dir=dummy_report_dir)
    # Add noise to the recent results to simulate the model performance change
    noise = np.random.uniform(-0.1, 0.1)
    dummy_result = {key: max(0, value - noise) for key, value in results_flat.items()}
    for name, metric in dummy_result.items():
        split, name = name.split("/")
        if name == "BinaryAUROC":
            dummy_report.log_quantitative_analysis(
                "performance",
                name=name,
                value=metric,
                metric_slice=split,
                pass_fail_thresholds=0.8,
                pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
            )
        elif name == "BinaryAccuracy":
            dummy_report.log_quantitative_analysis(
                "performance",
                name=name,
                value=metric,
                metric_slice=split,
                pass_fail_thresholds=0.6,
                pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
            )
        elif name == "BinaryPrecision":
            dummy_report.log_quantitative_analysis(
                "performance",
                name=name,
                value=metric,
                metric_slice=split,
                pass_fail_thresholds=0.6,
                pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
            )
        elif name == "BinaryRecall":
            dummy_report.log_quantitative_analysis(
                "performance",
                name=name,
                value=metric,
                metric_slice=split,
                pass_fail_thresholds=0.6,
                pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
            )
        elif name == "BinaryF1Score":
            dummy_report.log_quantitative_analysis(
                "performance",
                name=name,
                value=metric,
                metric_slice=split,
                pass_fail_thresholds=0.6,
                pass_fail_threshold_fns=lambda x, threshold: x >= threshold,
            )
    # Rename the report folder to the dummy date to simulate the time change
    dummy_report_path = dummy_report.export()
    date_dir = Path(dummy_report_path).parents[1]
    dummy_date = date.today() + relativedelta(months=-(6 * (i + 1)))
    new_dir = f"{dummy_report_dir}/{dummy_date}"
    if os.path.exists(new_dir):
        shutil.rmtree(new_dir)
    os.rename(date_dir, new_dir)

In [ ]:
# Collecting performance metrics from previous reports and current report
trends = get_metrics_trends(
    report_directory=dummy_report_dir,
    flat_results=results_flat,
    slice_names=["overall"],
)
shutil.rmtree(dummy_report_dir)

In [ ]:
# Plotting the performance over time
trends_plot = plotter.metrics_trends(trends)
report.log_plotly_figure(
    fig=trends_plot,
    caption="Performance over time",
    section_name="quantitative analysis",
)
trends_plot.show()

## Report Generation

Before generating the model card, let us document some of the details of the model and some considerations involved in developing and using the model.


Let's start with populating the model details section, which includes the following fields by default:
- description: A high-level description of the model and its usage for a general audience.
- version: The version of the model.
- owners: The individuals or organizations that own the model.
- license: The license under which the model is made available.
- citation: The citation for the model.
- references: Links to resources that are relevant to the model.
- path: The path to where the model is stored.
- regulatory_requirements: The regulatory requirements that are relevant to the model.

We can add additional fields to the model details section by passing a dictionary to the `log_from_dict` method and specifying the section name as `model_details`. You can also use the `log_descriptor` method to add a new field object with a `description` attribute to any section of the model card.

In [ ]:
report.log_from_dict(
    data={
        "description": "The model was trained on the Kaggle Heart Failure \
        Prediction Dataset to predict risk of heart failure.",
    },
    section_name="model_details",
)
report.log_version(
    version_str="0.0.1",
    date=str(date.today()),
    description="Initial Release",
)
report.log_owner(name="CyclOps Team", contact="vectorinstitute.github.io/cyclops/")
report.log_license(identifier="Apache-2.0")
report.log_reference(
    link="https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html"  # noqa: E501
)

Next, let's populate the considerations section, which includes the following fields by default:
- users: The intended users of the model.
- use_cases: The use cases for the model. These could be primary, downstream or out-of-scope use cases.
- fairness_assessment: A description of the benefits and harms of the model for different groups as well as the steps taken to mitigate the harms.
- ethical_considerations: The risks associated with using the model and the steps taken to mitigate them. This can be populated using the  `log_risk` method.



In [ ]:
report.log_from_dict(
    data={
        "users": [
            {"description": "Hospitals"},
            {"description": "Clinicians"},
        ]
    },
    section_name="considerations",
)
report.log_user(description="ML Engineers")
report.log_use_case(
    description="Predicting risk of heart failure.",
    kind="primary",
)
report.log_use_case(
    description="Predicting risk of pathologies and conditions other\
    than heart failure.",
    kind="out-of-scope",
)
report.log_fairness_assessment(
    affected_group="sex, age",
    benefit="Improved health outcomes for patients.",
    harm="Biased predictions for patients in certain groups (e.g. older patients) \
        may lead to worse health outcomes.",
    mitigation_strategy="We will monitor the performance of the model on these groups \
        and retrain the model if the performance drops below a certain threshold.",
)
report.log_risk(
    risk="The model may be used to make decisions that affect the health of patients.",
    mitigation_strategy="The model should be continuously monitored for performance \
        and retrained if the performance drops below a certain threshold.",
)

Once the model card is populated, you can generate the report using the `export` method. The report is generated in the form of an HTML file. A JSON file containing the model card data will also be generated along with the HTML file. By default, the files will be saved in a folder named `cyclops_reports` in the current working directory. You can change the path by passing a `output_dir` argument when instantiating the `ModelCardReport` class.

In [ ]:
report_path = report.export()
shutil.copy(f"{report_path}", ".")

You can view the generated HTML [report](./model_card.html).